In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
tf.random.set_seed(1234)
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

In [ ]:
import glob

folder_path = "/content/drive/MyDrive/data/falling"  # Replace with the actual path to your folder

# Use glob to get a list of all files in the folder
files = glob.glob(folder_path + "/*")

# Now, the 'files' variable contains a list of file paths in the specified folder
print(files)


['/content/drive/MyDrive/data/falling/falling14.csv', '/content/drive/MyDrive/data/falling/falling5.csv', '/content/drive/MyDrive/data/falling/falling1.csv', '/content/drive/MyDrive/data/falling/falling9.csv', '/content/drive/MyDrive/data/falling/falling10.csv', '/content/drive/MyDrive/data/falling/falling27.csv', '/content/drive/MyDrive/data/falling/falling23.csv', '/content/drive/MyDrive/data/falling/falling17.csv', '/content/drive/MyDrive/data/falling/falling22.csv', '/content/drive/MyDrive/data/falling/falling16.csv', '/content/drive/MyDrive/data/falling/falling26.csv', '/content/drive/MyDrive/data/falling/falling25.csv', '/content/drive/MyDrive/data/falling/falling18.csv', '/content/drive/MyDrive/data/falling/falling15.csv', '/content/drive/MyDrive/data/falling/falling12.csv', '/content/drive/MyDrive/data/falling/falling20.csv', '/content/drive/MyDrive/data/falling/falling19.csv', '/content/drive/MyDrive/data/falling/falling28.csv', '/content/drive/MyDrive/data/falling/falling21.c

In [ ]:
def parse_dataset(num_points=400):

    train_points = []
    train_labels = []
    class_map = {}
    folder_path = "/content/drive/MyDrive/data"
    folders = glob.glob(folder_path + "/*")

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files

        train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

        for f in train_files:


            # Use the random indices to select 200 random rows from the original array
            train_points.append(pd.read_csv(f).set_index('Unnamed: 0').sample(num_points).values)
            train_labels.append(i)

    return (
        np.array(train_points),
        np.array(train_labels),
        class_map,
    )

In [ ]:
NUM_POINTS = 150
NUM_CLASSES = 4
BATCH_SIZE = 1
train_points,train_labels,class_map = parse_dataset(NUM_POINTS)
data_train, data_val, labels_train, labels_val = train_test_split(train_points, train_labels, test_size=0.2, random_state=42)

processing class: falling
processing class: lying_down
processing class: sitting
processing class: standing


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/data')


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
import torch

from time import time
import numpy as np

def timeit(tag, t):
    print("{}: {}s".format(tag, time() - t))
    return time()

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.

    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst

    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist


def index_points(points, idx):
    """

    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    """
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]
    return new_points


def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    return centroids


def query_ball_point(radius, nsample, xyz, new_xyz):
    """
    Input:
        radius: local region radius
        nsample: max sample number in local region
        xyz: all points, [B, N, 3]
        new_xyz: query points, [B, S, 3]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    device = xyz.device
    B, N, C = xyz.shape
    _, S, _ = new_xyz.shape
    group_idx = torch.arange(N, dtype=torch.long).to(device).view(1, 1, N).repeat([B, S, 1])
    sqrdists = square_distance(new_xyz, xyz)
    group_idx[sqrdists > radius ** 2] = N
    group_idx = group_idx.sort(dim=-1)[0][:, :, :nsample]
    group_first = group_idx[:, :, 0].view(B, S, 1).repeat([1, 1, nsample])

    mask = group_idx == N
    group_idx[mask] = group_first[mask]
    return group_idx


def sample_and_group(npoint, radius, nsample, xyz, points, returnfps=False):
    """
    Input:
        npoint:
        radius:
        nsample:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, npoint, nsample, 3]
        new_points: sampled points data, [B, npoint, nsample, 3+D]
    """
    B, N, C = xyz.shape
    S = npoint
    fps_idx = farthest_point_sample(xyz, npoint) # [B, npoint, C]
    new_xyz = index_points(xyz, fps_idx)
    idx = query_ball_point(radius, nsample, xyz, new_xyz)
    grouped_xyz = index_points(xyz, idx) # [B, npoint, nsample, C]
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, C)

    if points is not None:
        grouped_points = index_points(points, idx)
        new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1) # [B, npoint, nsample, C+D]
    else:
        new_points = grouped_xyz_norm
    if returnfps:
        return new_xyz, new_points, grouped_xyz, fps_idx
    else:
        return new_xyz, new_points


def sample_and_group_all(xyz, points):
    """
    Input:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, 1, 3]
        new_points: sampled points data, [B, 1, N, 3+D]
    """
    device = xyz.device
    B, N, C = xyz.shape
    new_xyz = torch.zeros(B, 1, C).to(device)
    grouped_xyz = xyz.view(B, 1, N, C)
    if points is not None:
        new_points = torch.cat([grouped_xyz, points.view(B, 1, N, -1)], dim=-1)
    else:
        new_points = grouped_xyz
    return new_xyz, new_points


class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all):
        super(PointNetSetAbstraction, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        """
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        """
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 2, 1)

        if self.group_all:
            new_xyz, new_points = sample_and_group_all(xyz, points)
        else:
            new_xyz, new_points = sample_and_group(self.npoint, self.radius, self.nsample, xyz, points)
        # new_xyz: sampled points position data, [B, npoint, C]
        # new_points: sampled points data, [B, npoint, nsample, C+D]
        new_points = new_points.permute(0, 3, 2, 1) # [B, C+D, nsample,npoint]
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points =  F.relu(bn(conv(new_points)))

        new_points = torch.max(new_points, 2)[0]
        new_xyz = new_xyz.permute(0, 2, 1)
        return new_xyz, new_points


class PointNetSetAbstractionMsg(nn.Module):
    def __init__(self, npoint, radius_list, nsample_list, in_channel, mlp_list):
        super(PointNetSetAbstractionMsg, self).__init__()
        self.npoint = npoint
        self.radius_list = radius_list
        self.nsample_list = nsample_list
        self.conv_blocks = nn.ModuleList()
        self.bn_blocks = nn.ModuleList()
        for i in range(len(mlp_list)):
            convs = nn.ModuleList()
            bns = nn.ModuleList()
            last_channel = in_channel + 3
            for out_channel in mlp_list[i]:
                convs.append(nn.Conv2d(last_channel, out_channel, 1))
                bns.append(nn.BatchNorm2d(out_channel))
                last_channel = out_channel
            self.conv_blocks.append(convs)
            self.bn_blocks.append(bns)

    def forward(self, xyz, points):
        """
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        """
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 2, 1)

        B, N, C = xyz.shape
        S = self.npoint
        new_xyz = index_points(xyz, farthest_point_sample(xyz, S))
        new_points_list = []
        for i, radius in enumerate(self.radius_list):
            K = self.nsample_list[i]
            group_idx = query_ball_point(radius, K, xyz, new_xyz)
            grouped_xyz = index_points(xyz, group_idx)
            grouped_xyz -= new_xyz.view(B, S, 1, C)
            if points is not None:
                grouped_points = index_points(points, group_idx)
                grouped_points = torch.cat([grouped_points, grouped_xyz], dim=-1)
            else:
                grouped_points = grouped_xyz

            grouped_points = grouped_points.permute(0, 3, 2, 1)  # [B, D, K, S]
            for j in range(len(self.conv_blocks[i])):
                conv = self.conv_blocks[i][j]
                bn = self.bn_blocks[i][j]
                grouped_points =  F.relu(bn(conv(grouped_points)))
            new_points = torch.max(grouped_points, 2)[0]  # [B, D', S]
            new_points_list.append(new_points)

        new_xyz = new_xyz.permute(0, 2, 1)
        new_points_concat = torch.cat(new_points_list, dim=1)
        return new_xyz, new_points_concat


class PointNetFeaturePropagation(nn.Module):
    def __init__(self, in_channel, mlp):
        super(PointNetFeaturePropagation, self).__init__()
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv1d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm1d(out_channel))
            last_channel = out_channel

    def forward(self, xyz1, xyz2, points1, points2):
        """
        Input:
            xyz1: input points position data, [B, C, N]
            xyz2: sampled input points position data, [B, C, S]
            points1: input points data, [B, D, N]
            points2: input points data, [B, D, S]
        Return:
            new_points: upsampled points data, [B, D', N]
        """
        xyz1 = xyz1.permute(0, 2, 1)
        xyz2 = xyz2.permute(0, 2, 1)

        points2 = points2.permute(0, 2, 1)
        B, N, C = xyz1.shape
        _, S, _ = xyz2.shape

        if S == 1:
            interpolated_points = points2.repeat(1, N, 1)
        else:
            dists = square_distance(xyz1, xyz2)
            dists, idx = dists.sort(dim=-1)
            dists, idx = dists[:, :, :3], idx[:, :, :3]  # [B, N, 3]

            dist_recip = 1.0 / (dists + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm
            interpolated_points = torch.sum(index_points(points2, idx) * weight.view(B, N, 3, 1), dim=2)

        if points1 is not None:
            points1 = points1.permute(0, 2, 1)
            new_points = torch.cat([points1, interpolated_points], dim=-1)
        else:
            new_points = interpolated_points

        new_points = new_points.permute(0, 2, 1)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        return new_points

In [ ]:
class get_model(nn.Module):
    def __init__(self,num_class,normal_channel=False):
        super(get_model, self).__init__()
        in_channel = 6 if normal_channel else 3
        self.normal_channel = normal_channel
        self.sa1 = PointNetSetAbstraction(npoint=512, radius=0.2, nsample=32, in_channel=in_channel, mlp=[64, 64, 128], group_all=False)
        self.sa2 = PointNetSetAbstraction(npoint=128, radius=0.4, nsample=64, in_channel=128 + 3, mlp=[128, 128, 256], group_all=False)
        self.sa3 = PointNetSetAbstraction(npoint=None, radius=None, nsample=None, in_channel=256 + 3, mlp=[256, 512, 1024], group_all=True)
        self.fc1 = nn.Linear(1024, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.drop1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(256, num_class)

    def forward(self, xyz):
        B, _, _ = xyz.shape
        if self.normal_channel:
            norm = xyz[:, 3:, :]
            xyz = xyz[:, :3, :]
        else:
            norm = None
        l1_xyz, l1_points = self.sa1(xyz, norm)
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)
        x = l3_points.view(B, 1024)
        x = self.drop1(F.relu(self.bn1(self.fc1(x))))
        x = self.drop2(F.relu(self.bn2(self.fc2(x))))
        x = self.fc3(x)
        x = F.log_softmax(x, -1)


        return x, l3_points



class get_loss(nn.Module):
    def __init__(self):
        super(get_loss, self).__init__()

    def forward(self, pred, target):
        total_loss = F.nll_loss(pred, target)

        return total_loss

In [ ]:

from torch.utils.data import DataLoader, TensorDataset


In [ ]:
train_labels.shape

(364,)

In [ ]:

import numpy as np

# Assuming your data is in a variable called 'data'
# Reshape data and labels
train_points = np.transpose(data_train, (0, 2, 1))
data_val = np.transpose(data_val, (0, 2, 1))
data_val = torch.Tensor(data_val)
labels_val = torch.LongTensor(labels_val)


In [ ]:
# Convert NumPy arrays to PyTorch tensors
data = torch.Tensor(train_points)
labels = torch.LongTensor(labels_train)

# Create a TensorDataset to hold your data and labels
dataset = TensorDataset(data, labels)

# Define batch size
batch_size = 32

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(torch.Tensor(data_val), torch.LongTensor(labels_val))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
train_points.shape

(291, 3, 150)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def calculate_accuracy(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_data, batch_labels in data_loader:
            output, _ = model(batch_data)
            preds = output.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy


In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
# # Initialize your model and loss function
model = get_model(num_class=4)
loss_fn = get_loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20  # Increase the number of epochs
best_accuracy = 0.0

for epoch in range(num_epochs):
    # Training loop
    total_loss = 0.0
    model.train()
    for batch_data, batch_labels in dataloader:
        optimizer.zero_grad()
        output, _ = model(batch_data)
        loss = loss_fn(output, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(dataloader)}")

    # Validation loop
    total_val_loss = 0.0
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_data, batch_labels in val_dataloader:
            output, _ = model(batch_data)
            val_loss = loss_fn(output, batch_labels)
            total_val_loss += val_loss.item()

            preds = output.argmax(dim=1).cpu().numpy()
            labels = batch_labels.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch + 1}, Validation Loss: {total_val_loss / len(val_dataloader)}, Accuracy: {accuracy}")

    # Update the learning rate using the scheduler
    scheduler.step()

    # Save the model with the best validation accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

print("Training complete")


Epoch 1, Training Loss: 0.7790171474218368
Epoch 1, Validation Loss: 1.2601314981778462, Accuracy: 0.6164383561643836
Epoch 2, Training Loss: 0.3735837161540985
Epoch 2, Validation Loss: 2.2402714093526206, Accuracy: 0.6164383561643836
Epoch 3, Training Loss: 0.34875286668539046
Epoch 3, Validation Loss: 2.583195765813192, Accuracy: 0.6164383561643836
Epoch 4, Training Loss: 0.23896290510892867
Epoch 4, Validation Loss: 2.6638132333755493, Accuracy: 0.6164383561643836
Epoch 5, Training Loss: 0.2611801691353321
Epoch 5, Validation Loss: 2.483221928278605, Accuracy: 0.6164383561643836
Epoch 6, Training Loss: 0.24289973750710486
Epoch 6, Validation Loss: 1.7225550413131714, Accuracy: 0.6164383561643836
Epoch 7, Training Loss: 0.15136981159448623
Epoch 7, Validation Loss: 1.0305904150009155, Accuracy: 0.6301369863013698
Epoch 8, Training Loss: 0.11441920250654221
Epoch 8, Validation Loss: 0.5381400684515635, Accuracy: 0.821917808219178
Epoch 9, Training Loss: 0.15708277747035027
Epoch 9, V

In [ ]:
model = get_model(num_class=4)
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

TEST:

In [ ]:
model.eval()
test_points = []
test_labels = []
class_maps = {}
folder_path = "/content/drive/MyDrive/test"
folders = glob.glob(folder_path + "/*")

for i, folder in enumerate(folders):
    print("processing class: {}".format(os.path.basename(folder)))
    # store folder name with ID so we can retrieve later
    class_maps[i] = folder.split("/")[-1]
    # gather all files

    test_files = glob.glob(os.path.join(folder_path,folder)+"/*")

    for f in test_files:


        # Use the random indices to select 200 random rows from the original array
        test_points.append(pd.read_csv(f).set_index('Unnamed: 0').sample(120).values)
        test_labels.append(i)

processing class: sitting
processing class: standing
processing class: falling
processing class: lying_down


In [ ]:
test_points=np.array(test_points)
test_labels=np.array(test_labels)

In [ ]:
test_points = np.transpose(test_points, (0, 2, 1))

In [ ]:
data_test = torch.Tensor(test_points)
labels_test = torch.LongTensor(test_labels)

# Create a TensorDataset to hold your data and labels
dataset_test = TensorDataset(data_test, labels_test)

# Define batch size
batch_size = 32

# Create a DataLoader
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
tot_test_loss = 0.0
preds = []
labs = []

with torch.no_grad():
    for batch_data, batch_labels in dataloader_test:
        output, _ = model(batch_data)
        test_loss = loss_fn(output, batch_labels)
        tot_test_loss += test_loss.item()

        pred = output.argmax(dim=1).cpu().numpy()
        label = batch_labels.cpu().numpy()
        preds.extend(pred)
        labs.extend(label)

accuracys = accuracy_score(labs, preds)
print(f" Validation Loss: {tot_test_loss / len(dataloader_test)}, Accuracy: {accuracys}")


 Validation Loss: 3.99638831615448, Accuracy: 0.2037037037037037
